<a href="https://colab.research.google.com/github/xyhuang7215/2021VRDL-final/blob/main/Faster_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html &> /dev/null
!git clone https://github.com/open-mmlab/mmdetection.git &> /dev/null
%cd mmdetection
!pip install -r requirements/build.txt &> /dev/null
!pip install "git+https://github.com/open-mmlab/cocoapi.git#subdirectory=pycocotools" &> /dev/null
!pip install -v -e .  &> /dev/null

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"","key":""}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!mkdir dataset
%cd dataset
!kaggle competitions download -c the-nature-conservancy-fisheries-monitoring
!unzip train.zip &> /dev/null
!unzip test_stg1.zip &> /dev/null
!7z x ./test_stg2.7z &> /dev/null
%cd ..

In [ ]:
!pip install gdown==4.2.0
!gdown https://drive.google.com/drive/folders/10oIFqeDbVcF0zfKRwjat4z5eYr2Yrzd-? -O ./ --folder &> /dev/null

In [ ]:
# !mkdir checkpoints
# !wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_1x_coco/faster_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.378_20200504_180032-c5925ee5.pth \
#       -O checkpoints/pretrained.pth
# !python tools/train.py final/detection.py

In [ ]:
import csv
import json
import os
import cv2
from mmdet.apis import inference_detector, init_detector

# Choose to use a config and initialize the detector
config = 'final/detection.py'
# Setup a checkpoint file to load
checkpoint = 'final/epoch_12.pth'
# initialize the detector
model = init_detector(config, checkpoint, device='cuda:0')

load checkpoint from local path: final/epoch_12.pth


In [ ]:
def result_to_bbox(result):
    for bbox in result:
        if len(bbox) != 0 and bbox[0][-1] > 0.3:
            return bbox[0]

def save_crop_img(model, img_folder, target_folder):
    for img in os.listdir(img_folder):
        result = inference_detector(model, img_folder + img)
        bbox = result_to_bbox(result)
        image = cv2.imread(img_folder + img)
        if bbox is not None:
            x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3]), 
            image = image[y1:y2, x1:x2]
        cv2.imwrite(target_folder + img, image)

In [ ]:
!mkdir train_crop
!mkdir train_crop/ALB
!mkdir train_crop/BET
!mkdir train_crop/DOL
!mkdir train_crop/LAG
!mkdir train_crop/NoF
!mkdir train_crop/OTHER
!mkdir train_crop/SHARK
!mkdir train_crop/YFT

In [ ]:
save_crop_img(model, 'dataset/train/ALB/', 'train_crop/ALB/')
save_crop_img(model, 'dataset/train/BET/', 'train_crop/BET/')
save_crop_img(model, 'dataset/train/DOL/', 'train_crop/DOL/')
save_crop_img(model, 'dataset/train/LAG/', 'train_crop/LAG/')
save_crop_img(model, 'dataset/train/NoF/', 'train_crop/NoF/')
save_crop_img(model, 'dataset/train/OTHER/', 'train_crop/OTHER/')
save_crop_img(model, 'dataset/train/SHARK/', 'train_crop/SHARK/')
save_crop_img(model, 'dataset/train/YFT/', 'train_crop/YFT/')
!zip -r train_crop.zip train_crop

In [ ]:
!mkdir test_stg1
!mkdir test_stg2

In [ ]:
save_crop_img(model, 'dataset/test_stg2/', 'test_stg2/')
save_crop_img(model, 'dataset/test_stg1/', 'test_stg1/')
!zip -r test_stg1.zip test_stg1
!zip -r test_stg2.zip test_stg2